# fbprophet 모델을 활용한 인공지능 비트 트레이더 경진대회 베이스라인 코드

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install statsmodels==0.11.1
# !pip install fbprophet

# 1. Library Import

In [3]:
import numpy as np
import pandas as pd
import gc
import math
import os.path
import time
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm
import copy
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from fbprophet import Prophet
import warnings
import datetime
from plotly import tools 
import plotly.offline as offline 
import plotly.graph_objs as go 
warnings.filterwarnings("ignore")

# 2. 데이터 불러오기

## 1) read_csv

In [4]:
data_path = os.getenv('HOME') + '/ModuWay/data'
train_x_df = pd.read_csv(data_path  + "/train_x_df.csv")
train_y_df = pd.read_csv(data_path  + "/train_y_df.csv")
test_x_df = pd.read_csv(data_path  + "/test_x_df.csv")

* sample_id : 개별 샘플의 인덱스
* time : x_df는 0분 ~ 1379분, y_df는 0분 ~ 119분의 값을 갖습니다. 동일한 샘플 내 시간 정보
* coin_index : 10가지 종류의 코인에 대한 비식별화 인덱스 (0 ~9)
* open : open price
* high : high price
* low : low price
* close : close price
* volume : 거래량
* quote_av : quote asset volume
* trades : 거래 건 수
* tb_base_av : taker buy base asset volume
* tb_quote_av : taker buy quote asset volume

In [36]:
def get_macd(df, short=12, long=26, t=9): 
    
    # 입력받은 값이 dataframe이라는 것을 정의해줌 
    df = pd.DataFrame(df)
    
    # MACD 관련 수식 
    ma_12 = df.close.ewm(span=12).mean() # 단기(12) EMA(지수이동평균) 
    ma_26 = df.close.ewm(span=26).mean() # 장기(26) EMA 
    macd = ma_12 - ma_26 # MACD 
    macds = macd.ewm(span=9).mean() # Signal 
    macdo = macd - macds # Oscillator 
    
    df = df.assign(macd=macd, macds=macds, macdo=macdo).dropna() 
    
    return df

In [5]:
train_x_df

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av
0,0,0,7,1.010004,1.010004,1.009612,1.010004,8.382875e+05,43160.632812,451.157288,7.326834e+05,37725.183594
1,0,1,7,1.009808,1.009808,1.009808,1.009808,1.622420e+05,8352.220703,39.231071,0.000000e+00,0.000000
2,0,2,7,1.009808,1.010200,1.009808,1.010200,1.664967e+04,857.377808,58.846603,1.664967e+04,857.377808
3,0,3,7,1.010200,1.011181,1.010200,1.011181,2.586971e+06,133310.343750,431.541779,2.189147e+06,112811.046875
4,0,4,7,1.010985,1.010985,1.010200,1.010200,1.129996e+06,58216.867188,176.539810,0.000000e+00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
10159555,7361,1375,8,1.000668,1.001669,1.000626,1.001502,9.180907e+00,2203.059082,2.245034,6.229020e+00,1494.727417
10159556,7361,1376,8,1.001627,1.001920,1.000960,1.001294,7.963097e+00,1911.151611,2.211651,3.056139e+00,733.490601
10159557,7361,1377,8,1.001294,1.001461,1.000584,1.000668,3.849893e+00,923.610718,1.260224,2.284546e+00,548.042297
10159558,7361,1378,8,1.000709,1.000751,1.000042,1.000042,1.337402e+00,320.624756,0.826239,5.164965e-01,123.819839


In [6]:
grouped = pd.DataFrame(train_x_df.groupby(train_x_df['sample_id'])['close'].rolling(window = 7, min_periods=1).mean())
grouped

close
sample_id                   
0         0         1.010004
          1         1.009906
          2         1.010004
          3         1.010298
          4         1.010279
...                      ...
7361      10159555  0.999350
          10159556  0.999815
          10159557  1.000101
          10159558  1.000364
          10159559  1.000656

[10159560 rows x 1 columns]

In [9]:
train_x_df.loc[:,'sma7'] = grouped['close'].values

In [11]:
grouped = pd.DataFrame(train_x_df.groupby(train_x_df['sample_id'])['close'].rolling(window = 25, min_periods=1).mean())
grouped

close
sample_id                   
0         0         1.010004
          1         1.009906
          2         1.010004
          3         1.010298
          4         1.010279
...                      ...
7361      10159555  0.998212
          10159556  0.998289
          10159557  0.998314
          10159558  0.998307
          10159559  0.998346

[10159560 rows x 1 columns]

In [12]:
train_x_df.loc[:,'sma25'] = grouped['close'].values

In [39]:
tmp = get_macd(train_x_df['close'])

tmp.head()

len(tmp)

10159560

In [35]:
grouped = pd.DataFrame(train_x_df.groupby(train_x_df['sample_id'])get_macd(train_x_df['close']))
grouped

AttributeError: 'SeriesGroupBy' object has no attribute 'get_macd'

In [ ]:
train_x_df

In [ ]:
grouped.reset_index(inplace=True)

In [ ]:
grouped.head(1781)

In [ ]:
grouped['close'].head(1781)

In [ ]:
train_x_df = get_macd(train_x_df) 

# 데이터 확인 
train_x_df

In [ ]:
train_y_df

In [ ]:
test_x_df['7_SMA'] = test_x_df['close'].rolling(window = 7, min_periods=1).mean()

In [ ]:
test_x_df

In [ ]:
# create 7 mins simple moving average column
train_y_df['7_SMA'] = train_y_df['close'].rolling(window = 7, min_periods=1).mean()
# create 25 mins simple moving average column
train_y_df['25_SMA'] = train_y_df['close'].rolling(window = 25,  min_periods=1).mean()

train_y_df = get_macd(train_y_df) 

# 데이터 확인 
train_y_df

In [ ]:
test_x_df

In [ ]:
# create 7 mins simple moving average column
test_x_df['7_SMA'] = test_x_df['close'].rolling(window = 7, min_periods=1).mean()
# create 25 mins simple moving average column
test_x_df['25_SMA'] = test_x_df['close'].rolling(window = 25,  min_periods=1).mean()

test_x_df = get_macd(test_x_df) 

# 데이터 확인 
test_x_df

## 2) numpy arrary로 변환하기

In [ ]:
def df2d_to_array3d(df_2d):
    # 입력 받은 2차원 데이터 프레임을 3차원 numpy array로 변경하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    sample_index = df_2d.sample_id.value_counts().index
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    return array_3d

In [ ]:
train_x_array = df2d_to_array3d(train_x_df)
train_y_array = df2d_to_array3d(train_y_df)
test_x_array = df2d_to_array3d(test_x_df)

In [ ]:
sma7 = []

for i in train_x_array[:, 2]:
    sma7 = train_x_array[:, train_x_array[:, 2] <= i][:, 6].rolling(window = 7, min_periods=1).mean()
    sma7.append(sma7)

In [ ]:
print(f'''
train_x_array {train_x_array.shape}
train_y_array {train_y_array.shape}
test_x_array {test_x_array.shape}
''')

In [ ]:
train_x_array[10, :]

## 3) 샘플 확인하기

## MACD 시각화 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as ticker
import numpy as np

In [ ]:
idx = 1000
index = train_x_df[train_x_df['sample_id']==idx].time.tolist()
type(index), len(index)

In [ ]:
# loc 쓰기 귀찮아서 데이터 프레임으로.. 그냥 .. 때려..넣기? => 나중에 수정
sample_x_df = pd.DataFrame()
sample_x_df = train_x_df[['sample_id','7_SMA', '25_SMA','macd','macdo','macds']]
sample_x_df.head()

In [ ]:
sma7_lst = sample_x_df['7_SMA'].loc[sample_x_df['sample_id']==idx].tolist()
sma25_lst = sample_x_df['25_SMA'].loc[sample_x_df['sample_id']==idx].tolist()

In [ ]:
print(len(sma25_lst) == len(sma7_lst)) # 차원 일치 여부 확인 
print(type(index) == type(sma7_lst)) # 형 일치 여부 확인 

In [ ]:
index[0:2], sma7_lst[0:2] # 값 확인

In [ ]:
# Main chart - SMA_7 and SMA_25
fig, ax = plt.subplots(figsize=(12,5))

ax.set_title('SMA_7 v.s. SMA_25', fontsize=20)

ax.plot(index, sma7_lst, 'b', lw=1, label='SMA_7')
ax.plot(index, sma25_lst, 'r', lw=1, label='SMA_25')
ax.legend(loc='best')

fig.tight_layout()

In [ ]:
macd_lst = sample_x_df['macd'].loc[sample_x_df['sample_id']==idx].tolist()
macds_lst = sample_x_df['macds'].loc[sample_x_df['sample_id']==idx].tolist()

In [ ]:
# index chart - MACD and MACDs
fig, ax = plt.subplots(figsize=(12,5))

ax.set_title('MACD v.s. MACD Sinal', fontsize=20)

ax.plot(index, macd_lst, 'b', lw=1, label='macd')
ax.plot(index, macds_lst, 'r', lw=1, label='macds')
ax.legend(loc='best')

fig.tight_layout()

In [ ]:
macdo_lst = sample_x_df['macdo'].loc[sample_x_df['sample_id']==idx].tolist()

In [ ]:
# index chart - MACDo
fig, ax = plt.subplots(figsize=(12,5))

ax.set_title('MACD Oscilator', fontsize=20)

ax.bar(index, [0 if i > 0 else i for i in macdo_lst], lw=1, label='macdo(+)')
ax.bar(index, [0 if i < 0 else i for i in macdo_lst], lw=1, label='macdo(-)')
ax.legend(loc='best')

fig.tight_layout()

In [ ]:
## MACD 시각화 

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as ticker
import numpy as np

idx = 1000
index = train_x_df[train_x_df['sample_id']==idx].time.tolist()
type(index), len(index)

# loc 쓰기 귀찮아서 데이터 프레임으로.. 그냥 .. 때려..넣기? => 나중에 수정
sample_x_df = pd.DataFrame()
sample_x_df = train_x_df[['sample_id','7_SMA', '25_SMA','macd','macdo','macds']]
sample_x_df.head()

sma7_lst = sample_x_df['7_SMA'].loc[sample_x_df['sample_id']==idx].tolist()
sma25_lst = sample_x_df['25_SMA'].loc[sample_x_df['sample_id']==idx].tolist()

print(len(sma25_lst) == len(sma7_lst)) # 차원 일치 여부 확인 
print(type(index) == type(sma7_lst)) # 형 일치 여부 확인 

index[0:2], sma7_lst[0:2] # 값 확인

# Main chart - SMA_7 and SMA_25
fig, ax = plt.subplots(figsize=(12,5))

ax.set_title('SMA_7 v.s. SMA_25', fontsize=20)

ax.plot(index, sma7_lst, 'b', lw=1, label='SMA_7')
ax.plot(index, sma25_lst, 'r', lw=1, label='SMA_25')
ax.legend(loc='best')

fig.tight_layout()

macd_lst = sample_x_df['macd'].loc[sample_x_df['sample_id']==idx].tolist()
macds_lst = sample_x_df['macds'].loc[sample_x_df['sample_id']==idx].tolist()

# index chart - MACD and MACDs
fig, ax = plt.subplots(figsize=(12,5))

ax.set_title('MACD v.s. MACD Sinal', fontsize=20)

ax.plot(index, macd_lst, 'b', lw=1, label='macd')
ax.plot(index, macds_lst, 'r', lw=1, label='macds')
ax.legend(loc='best')

fig.tight_layout()

macdo_lst = sample_x_df['macdo'].loc[sample_x_df['sample_id']==idx].tolist()

# index chart - MACDo
fig, ax = plt.subplots(figsize=(12,5))

ax.set_title('MACD Oscilator', fontsize=20)

ax.bar(index, [0 if i > 0 else i for i in macdo_lst], lw=1, label='macdo(+)')
ax.bar(index, [0 if i < 0 else i for i in macdo_lst], lw=1, label='macdo(-)')
ax.legend(loc='best')

fig.tight_layout()

In [ ]:
def plot_series(x_series, y_series):
    #입력 series와 출력 series를 연속적으로 연결하여 시각적으로 보여주는 코드 입니다.
    plt.plot(x_series, label = 'input_series')
    plt.plot(np.arange(len(x_series), len(x_series)+len(y_series)),
             y_series, label = 'output_series')
    plt.axhline(1, c = 'red')
    plt.legend()

In [ ]:
# sample_id 1012에 해당하는 sample의 분단위 시가 변동 정보 시각화
idx = 1012
plot_series(train_x_array[idx,:,1], train_y_array[idx,:,1]) # sample_id, time, 나머지 컬럼 정보들
plt.show()

# 3. 모델 훈련하기

## 1) train 샘플 훈련하기

In [ ]:
start_time = '2021-01-31 00:00:00'
start_dt = datetime.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')

In [ ]:
idx = 1121
# train data 중 sample_id 1121에 해당하는 x_series로 arima 모델을 학습한 후
# y_sereis를 추론
x_series = train_x_array[idx,:,1]
y_series = train_y_array[idx,:,1]

In [ ]:
x_df = pd.DataFrame()
x_df['ds'] = [start_dt + datetime.timedelta(minutes = time_min) for time_min in np.arange(1, x_series.shape[0]+1).tolist()]
x_df['y'] = x_series.tolist()

In [ ]:
x_df.head()

In [ ]:
prophet = Prophet(seasonality_mode='multiplicative', 
                  yearly_seasonality=False,
                  weekly_seasonality=False, daily_seasonality=False)
prophet.fit(x_df)

In [ ]:
# 120분 테스트 데이터를 예측합니다.
future_data = prophet.make_future_dataframe(periods=120, freq='min')
forecast_data = prophet.predict(future_data)
forecast_data[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(5)

In [ ]:
# 만들어진 데이터를 확인합니다.
fig1 = prophet.plot(forecast_data)

In [ ]:
# 데이터의 추세를 확인합니다.
fig2 = prophet.plot_components(forecast_data)

In [ ]:
pred_y = forecast_data.yhat.values[-120:]
pred_y_lower = forecast_data.yhat_lower.values[-120:]
pred_y_upper = forecast_data.yhat_upper.values[-120:]

plot_series(x_series, y_series)
plt.plot(np.arange(1380, 1380+120), pred_y, label = 'prediction')
plt.plot(np.arange(1380, 1380+120), pred_y_lower, label = 'prediction_lower')
plt.plot(np.arange(1380, 1380+120), pred_y_upper, label = 'prediction_upper')
plt.legend()
plt.show()

## 2) validation set 훈련하기

### <1> validation 데이터 생성하기

In [ ]:
# train 데이터 상단의 300개 샘플로 validation set를 구성하여 학습 및 추론
valid_x_df = train_x_df[train_x_df.sample_id < 300]
valid_y_df = train_y_df[train_y_df.sample_id < 300]

valid_x_array = df2d_to_array3d(valid_x_df)
valid_y_array = df2d_to_array3d(valid_y_df)
valid_pred_array = np.zeros([300, 120])

### <2> 학습 및 추론하기

In [ ]:

for idx in tqdm(range(valid_x_array.shape[0])):
    try:
        x_series = valid_x_array[idx,:,1]

        x_df = pd.DataFrame()
        x_df['ds'] = [start_dt + datetime.timedelta(minutes = time_min) for time_min in np.arange(1, x_series.shape[0]+1).tolist()]
        x_df['y'] = x_series.tolist()

        prophet = Prophet(seasonality_mode='multiplicative', 
                  yearly_seasonality=False,
                  weekly_seasonality=False, daily_seasonality=False)
        prophet.fit(x_df)

        # 120분 테스트 데이터를 예측합니다.
        future_data = prophet.make_future_dataframe(periods=120, freq='min')
        forecast_data = prophet.predict(future_data)

        pred_y = forecast_data.yhat.values[-120:]
        pred_y_lower = forecast_data.yhat_lower.values[-120:]
        pred_y_upper = forecast_data.yhat_upper.values[-120:]

        valid_pred_array[idx,:] = pred_y
    except:
        print(idx, " 샘플은 수렴하지 않습니다.")
        pass

### <3> valid_pred_array 로부터 buy_quantity, sell_time 구하기

In [ ]:
def array_to_submission(x_array, pred_array):
    # 입력 x_arrry와 출력 pred_arry를 통해서 
    # buy_quantitiy와 sell_time을 결정
    submission = pd.DataFrame(np.zeros([pred_array.shape[0],2], np.int64),
                columns = ['buy_quantity', 'sell_time'])
    submission = submission.reset_index()
    submission.loc[:, 'buy_quantity'] = 0.1
    
    buy_price = []
    for idx, sell_time in enumerate(np.argmax(pred_array, axis = 1)):
        buy_price.append(pred_array[idx, sell_time])
    buy_price = np.array(buy_price)
    # 115% 이상 상승한하고 예측한 sample에 대해서만 100% 매수
    submission.loc[:, 'buy_quantity'] = (buy_price > 1.15) * 1
    # 모델이 예측값 중 최대 값에 해당하는 시간에 매도
    submission['sell_time'] = np.argmax(pred_array, axis = 1)
    submission.columns = ['sample_id','buy_quantity', 'sell_time']
    return submission

In [ ]:
valid_submission = array_to_submission(valid_x_array, valid_pred_array)

In [ ]:
valid_submission

In [ ]:
# 전체 300가지 sample에 대해
# 5가지 case에서 115% 이상 상승한다고 추론함.
valid_submission.buy_quantity.value_counts()

### <4> 투자 후 금액 계산하기

In [ ]:
def df2d_to_answer(df_2d):
    # valid_y_df로부터
    # open 가격 정보가 포함된
    # [샘플 수, 120분] 크기의 
    # 2차원 array를 반환하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    sample_index = df_2d.sample_id.value_counts().index
    array_2d = df_2d.open.values.reshape([sample_size, time_size])
    sample_index = list(sample_index)
    return array_2d, sample_index


def COIN(y_df, submission, df2d_to_answer = df2d_to_answer):
    # 2차원 데이터프레임에서 open 시점 데이터만 추출하여 array로 복원
    # sample_id정보를 index에 저장
    y_array, index = df2d_to_answer(y_df)
    
    # index 기준으로 submission을 다시 선택
    submission = submission.set_index(submission.columns[0])
    submission = submission.iloc[index, :]    
    
    # 초기 투자 비용은 10000 달러
    total_momey      = 10000 # dolors
    total_momey_list = []
    
    # 가장 처음 sample_id값
    start_index = submission.index[0]
    for row_idx in submission.index:
        sell_time  = submission.loc[row_idx, 'sell_time']
        buy_price  = y_array[row_idx - start_index, 0]
        sell_price = y_array[row_idx - start_index, sell_time]
        buy_quantity = submission.loc[row_idx, 'buy_quantity'] * total_momey
        residual = total_momey - buy_quantity
        ratio = sell_price / buy_price
        total_momey = buy_quantity * ratio * 0.9995 * 0.9995 + residual        
        total_momey_list.append(total_momey)
        
    return total_momey, total_momey_list

In [ ]:
total_momey, total_momey_list = COIN(valid_y_df,
                                     valid_submission)

In [ ]:
# 투자 후 금액
print(total_momey)

In [ ]:
# 투자 히스토리
plt.plot(total_momey_list)
plt.title("history")
plt.show()

# 4. test 데이터 학습하고 추론하기

In [ ]:
test_pred_array = np.zeros([test_x_array.shape[0],
                           120])
for idx in tqdm(range(test_x_array.shape[0])):
    try:
        x_series = test_x_array[idx,:,1]

        x_df = pd.DataFrame()
        x_df['ds'] = [start_dt + datetime.timedelta(minutes = time_min) for time_min in np.arange(1, x_series.shape[0]+1).tolist()]
        x_df['y'] = x_series.tolist()

        prophet = Prophet(seasonality_mode='multiplicative', 
                  yearly_seasonality=False,
                  weekly_seasonality=False, daily_seasonality=False)
        prophet.fit(x_df)

        # 120분 테스트 데이터를 예측합니다.
        future_data = prophet.make_future_dataframe(periods=120, freq='min')
        forecast_data = prophet.predict(future_data)

        pred_y = forecast_data.yhat.values[-120:]
        pred_y_lower = forecast_data.yhat_lower.values[-120:]
        pred_y_upper = forecast_data.yhat_upper.values[-120:]

        test_pred_array[idx,:] = pred_y
    except:
        print(idx, " 샘플은 수렴하지 않습니다.")
        pass

## 추론한 test_pred_array를 바탕으로 submission df 생성하기

In [ ]:
submission = array_to_submission(test_x_array, test_pred_array)

In [ ]:
submission.to_csv(data_path+"/baseline_submission_proh.csv", index = False)